In [3]:
import json
import re
import csv
import requests
import scrapy


base_url = "http://goheels.com/roster.aspx?path=mbball"

headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36"}



resp = requests.get(base_url,headers=headers)
body_bytes = resp.content
body_str = body_bytes.decode("utf-8")
sel = scrapy.Selector(text=body_str)
table = sel.css('.sidearm-table-grid-template-1-breakdown-large')

record_dict = {}  # map the name to its record, which is a nother dict
personas = table.css('tbody:nth-child(3) tr')  # get all information
for persona in personas:
    info_record = {}  # create an empty dict in the record_dict
    name_obj = persona.css('.sidearm-table-player-name')
    name = name_obj.xpath('string()').extract()[0]  # get name in .name class
    href = 'http://goheels.com' + name_obj.xpath('a/@href').extract()[0]  # get href in .name class

    resppicture = requests.get(href, headers={
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36"})
    bodypicture_bytes = resppicture.content
    body_str_pic = bodypicture_bytes.decode("utf-8")
    selpicture = scrapy.Selector(text=body_str_pic)

    picture = selpicture.css('.sidearm-roster-player-image').xpath('img/@src').extract()[0]
    bio = selpicture.css('#sidearm-roster-player-bio').xpath("string()").extract()[0]

    info_record["name"] = name
    info_record["href"] = href
    info_record["number"] = persona.css("td:nth-child(1)").xpath("string()").extract()[0]
    info_record["position"] = persona.css("td:nth-child(3)").xpath("string()").extract()[0]
    info_record["height"] = persona.css("td:nth-child(4)").xpath("string()").extract()[0]
    info_record["weight"] = persona.css("td:nth-child(5)").xpath("string()").extract()[0]
    info_record["class"] = persona.css("td:nth-child(6)").xpath("string()").extract()[0]
    info_record["hometown_highschool"] = persona.css("td:nth-child(7)").xpath("string()").extract()[0]
    info_record["picture"] = 'http://goheels.com' + picture
    
    info_record["bio"] = bio.replace("\n", "").replace("\r", "").strip()

    record_dict[name] = info_record


output_file = open('records.json', 'w+')  # create json file
json.dump(record_dict, output_file)



In [ ]:
#################################
# following code: scrape each player's STATS info 

In [9]:
rp_search = re.compile(r"(\d+)$")
id_search = re.compile(r"\$\.getJSON\(\"/services/responsive-roster-bio\.ashx\", {.*?player_id: (\d+) }")
#group id number 

id_numbers = [] #use for debug

for key,rec in record_dict.items():
    ################################
    ## Deal with stats
    cur_href = rec["href"]   #cur refers to current

    cur_resp = requests.get(cur_href, headers=headers)
    cur_body = cur_resp.content.decode("utf-8")

    cur_playernum = rec["number"]
    
    cur_playerid = id_search.search(cur_body).group(1)
    id_numbers.append(cur_playerid) #put the newly found playerid to the list "id_numbers"

    cur_rp = rp_search.search(cur_href).group(1) #rp refers to rp id
    cur_stat_url = 'http://goheels.com/services/responsive-roster-bio.ashx?type=stats&rp_id={}&path=mbball&year=2017&player_id={}'.format(cur_rp, cur_playerid)

    stat_resp = requests.get(cur_stat_url,headers=headers)
    
    stat_content =  stat_resp.content.decode("utf-8")

    stat_json = json.loads(stat_content) #stat_json is a dictionary
    #stat_content is a string in json format, so needs to load string json
    
    cur_stat_html = stat_json["current_stats"]
    #current_stats is a key named by the website author, key to html string

    ## now we're organizing those stats and categorize them, saving them to a csv

    if cur_playerid=='0':
        continue
        #this is for the special person who doesn't have any stat, his id is 0

    attribute_list = []  # make a list of all the category names
    sel_stat = scrapy.Selector(text=cur_stat_html).css("table")  # get all information
    stat_attributes = sel_stat.css("thead tr th")
    for attr in stat_attributes:
        attribute_list.append(attr.xpath("string()").extract()[0])

    f = open("current_stats_of_{}.csv".format(cur_playernum),'w+') # new a csv and open it
    stat_writer = csv.writer(f)
    stat_writer.writerow(attribute_list)  #write category names into 1st row of the csv

    stat_rows = sel_stat.css("tbody tr")
    for stat_row in stat_rows:
        cur_row=[]
        cur_row.append(stat_row.css("td:nth-child(1)").xpath("string()").extract()[0]) 
        #put the dates to the list
        
        cur_row.append(stat_row.css("th").xpath("string()").extract()[0])
        #put the name to the list
        
        for i in stat_row.xpath('td[@class="text-center"]'):
            cur_row.append(i.xpath("string()").extract()[0])
            #put other stats to the list
        stat_writer.writerow(cur_row)
        #write list to the csv file
    

    f.close()   # close the csv

   

In [ ]:
    ################################
  #so we get a json file includes all the basic info (scraped by the first part of the code)
 # and several csv file, each includes a player's STATs (scraped by the second part of the code)
    #and we can clean them in panda 
    # WoW!!!!!! XD DONE!!